In [1]:
import os
os.chdir('D:\Python_projects\Insurance_Prediction')



In [44]:

from InsurancePremiumPrediction.entity.artifact_entity import DataIngestionArtifact,\
    DataValidationArtifact,DataTransformationArtifact, ModelTrainerArtifact
from InsurancePremiumPrediction.entity.config_entity import ModelTrainerConfig
from InsurancePremiumPrediction.util.util import load_numpy_array_data
import os,sys

from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics._regression import r2_score
from xgboost import XGBRegressor
import pickle


In [45]:
os.chdir('D:\Python_projects\Insurance_Prediction')

In [4]:
model_dict={}
linear_model=LinearRegression()
decision_tree_model=DecisionTreeRegressor()
knn_model=KNeighborsRegressor()
gradient_booster=GradientBoostingRegressor()
random_booster=RandomForestRegressor()
xgb_regressor_model=XGBRegressor()
model_dict['linear_model']=linear_model
model_dict['decision_tree_model']=decision_tree_model
model_dict['knn_model']=knn_model
model_dict['gradient_booster']=gradient_booster
model_dict['random_booster']=random_booster
model_dict['xgb_regressor_model']=xgb_regressor_model

        

In [8]:
#gradient_booster.set_params(max_depth=2, min_samples_split=7, random_state=100,alpha=0.1)

GradientBoostingRegressor(alpha=0.1, max_depth=2, min_samples_split=7,
                          random_state=100)

In [5]:
model_dict

{'linear_model': LinearRegression(),
 'decision_tree_model': DecisionTreeRegressor(),
 'knn_model': KNeighborsRegressor(),
 'gradient_booster': GradientBoostingRegressor(),
 'random_booster': RandomForestRegressor(),
 'xgb_regressor_model': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, reg_lambda=None, ...)}

In [6]:
train_array_path=r'D:\Python_projects\Insurance_Prediction\InsurancePremiumPrediction\artifact\data_transformation\22-07-19-07-52-22\transformed_data\train\insurance.npz'
test_array_path=r'D:\Python_projects\Insurance_Prediction\InsurancePremiumPrediction\artifact\data_transformation\22-07-19-07-52-22\transformed_data\test\insurance.npz'

In [7]:
train_array=load_numpy_array_data(train_array_path)
test_array=load_numpy_array_data(test_array_path)
X_train,Y_train,X_test,Y_test=train_array[:,:-1],train_array[:,-1],test_array[:,:-1],test_array[:,-1]

#X_train,X_test,Y_train,Y_test=train_test_split(X_train,Y_train,random_state=42,test_size=.25)

In [8]:
model_score_set={}
for i in model_dict.values():
    i.fit(X_train,Y_train)
    Y_predict=i.predict(X_test)
    model_score_set[i]=[i.score(X_train,Y_train),r2_score(Y_test,Y_predict)]
    

In [9]:
model_score_set

{LinearRegression(): [0.7564369419272546, 0.7215214663841498],
 DecisionTreeRegressor(): [0.9983784082272196, 0.6225056637488888],
 KNeighborsRegressor(): [0.8526318701881948, 0.7348962189518002],
 GradientBoostingRegressor(): [0.9081263946076713, 0.8218153275621483],
 RandomForestRegressor(): [0.9772099562759671, 0.8123474161761627],
 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=

In [10]:
x=list(model_score_set.values())
import numpy as np
x=np.array(x)
max=0
best_model_dict={}
for i in range(6):
    if max <x[i][1]:
        max=x[i][1]

In [11]:
max

0.8218153275621483

In [12]:
gradient_booster.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'ls',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [13]:
para_grid_gradient={
    'min_samples_leaf':[1,3,7],
    'max_depth': [1,2,3,4],
    'random_state': range(100,500,50),
    'alpha': [0.1,0.2,0.9],
    }


In [14]:
grid_cv_gradient=GridSearchCV(error_score='raise',estimator=gradient_booster,param_grid=para_grid_gradient,cv=2,n_jobs=-1,verbose=1)

In [15]:
grid_cv_gradient.fit(X_train,Y_train)

Fitting 2 folds for each of 288 candidates, totalling 576 fits


GridSearchCV(cv=2, error_score='raise', estimator=GradientBoostingRegressor(),
             n_jobs=-1,
             param_grid={'alpha': [0.1, 0.2, 0.9], 'max_depth': [1, 2, 3, 4],
                         'min_samples_leaf': [1, 3, 7],
                         'random_state': range(100, 500, 50)},
             verbose=1)

In [16]:
x=grid_cv_gradient.best_params_

In [17]:
print(x)

{'alpha': 0.1, 'max_depth': 2, 'min_samples_leaf': 7, 'random_state': 100}


In [31]:
GradientBoostingRegressor(**grid_cv_gradient.best_params_)

GradientBoostingRegressor(alpha=0.1, max_depth=2, min_samples_leaf=7,
                          random_state=100)

In [32]:
gradient_booster.set_params(**grid_cv_gradient.best_params_)

GradientBoostingRegressor(alpha=0.1, max_depth=2, min_samples_leaf=7,
                          random_state=100)

In [36]:
gradient_booster.fit(X_train,Y_train)

GradientBoostingRegressor(alpha=0.1, max_depth=2, min_samples_leaf=7,
                          random_state=100)

In [37]:
gradient_booster.score(X_test,Y_test)


0.8375451809215317

2

In [132]:
y_predict1=grid_cv_gradient.predict(X_test)

In [133]:
r2_score(Y_test,y_predict1)

0.8375451809215317

0.8348289232587586

In [24]:
gradient_booster_new=GradientBoostingRegressor(max_depth=2, min_samples_split=7, random_state=100,alpha=0.1)

In [25]:
gradient_booster_new.fit(X_train,Y_train)
gradient_booster_new.score(X_train,Y_train)

0.8935007317594853

In [26]:
Y_predict_=gradient_booster_new.predict(X_test)

In [27]:
gradient_booster_new.score(X_test,Y_test)

0.8523938736122534

In [28]:
gradient_booster_new.alpha

0.1

In [29]:
r2_score(Y_test,Y_predict_)

0.8523938736122534

In [30]:
y_predict_validation=gradient_booster_new.predict(x_test_validation)

In [31]:
r2_score(y_test_validation,y_predict_validation)

0.8350460382636088

In [42]:
from InsurancePremiumPrediction.logger import logging
train=load_numpy_array_data(file_path=train_array_path)
test=load_numpy_array_data(file_path=test_array_path)

x_train=train[:,:-1]
y_train=train[:,-1]
x_test=test[:,:-1]
y_test=test[:,-1]        

        
models_r2_scores=[]
        
linear_reg = LinearRegression()
linear_reg.fit(x_train,y_train)
y_predict=linear_reg.predict(x_test)
linear_reg_r2_score = r2_score(y_test, y_predict)
models_r2_scores.append(linear_reg_r2_score)
logging.info(f"Linear Regression r2 score : {linear_reg_r2_score }")

decision_tree = DecisionTreeRegressor()
decision_tree.fit(x_train, y_train)
y_pred1 = decision_tree.predict(x_test)
decision_tree_r2_score = r2_score(y_test, y_pred1)
models_r2_scores.append(decision_tree_r2_score)
logging.info(f"Linear Regression r2 score : {decision_tree_r2_score }")

random_forest_reg =RandomForestRegressor()
random_forest_reg.fit(x_train, y_train)                        
y_pred2 = random_forest_reg.predict(x_test)
random_forest_r2_score = r2_score(y_test, y_pred2)
models_r2_scores.append(random_forest_r2_score)       
logging.info(f"Random Forest r2 score : {random_forest_r2_score }")

gradient_boosting_reg =GradientBoostingRegressor()
gradient_boosting_reg.fit(x_train, y_train)
y_pred3 = gradient_boosting_reg.predict(x_test)
gradient_boosting_r2_score = r2_score(y_test, y_pred3)
models_r2_scores.append(gradient_boosting_r2_score)
logging.info(f"Gradient Boosting r2 score : { gradient_boosting_r2_score }")

xgb_reg =XGBRegressor()
xgb_reg.fit(x_train, y_train)
y_pred4 = xgb_reg.predict(x_test)
xgb_reg_r2_score = r2_score(y_test, y_pred4)
models_r2_scores.append(xgb_reg_r2_score) 
logging.info(f"XGBRegressor r2 score : { xgb_reg_r2_score }")




In [43]:
type(random_forest_r2_score)

numpy.float64

In [44]:
gradient_boosting_r2_score

0.8220232921655246

In [45]:
models_r2_scores

[0.7215214663841498,
 0.6211800852435183,
 0.8101586935823564,
 0.8220232921655246,
 0.7470240997712023]

In [49]:
max=0
index=0
for i in range(5):
    if models_r2_scores[i] > max:
        max=models_r2_scores[i]
        index=i


In [51]:

print(index, max)

3 0.8220232921655246


In [40]:
abs(.908-.821)

0.08700000000000008

In [48]:
def training_models(x_train,y_train,x_test,y_test):
        try:
            X_train,Y_train,X_test,Y_test=x_train,y_train,x_test,y_test

            model_dict={}
            linear_model=LinearRegression()
            decision_tree_model=DecisionTreeRegressor()
            random_forest=RandomForestRegressor()
            gradient_booster=GradientBoostingRegressor()

            model_dict['linear_model']=linear_model
            model_dict['decision_tree_model']=decision_tree_model
            model_dict['random_forest']=random_forest
            model_dict['gradient_booster']=gradient_booster

            model_score_set={}
            for i in model_dict.values():
                i.fit(X_train,Y_train)
                Y_predict=i.predict(X_test)
                model_score_set[i]=[i.score(X_train,Y_train),r2_score(Y_test,Y_predict)]
            return model_dict,model_score_set
        except Exception as e:
            raise Exception


In [49]:
train_array_path=r'D:\Python_projects\Insurance_Prediction\InsurancePremiumPrediction\artifact\data_transformation\22-07-19-07-52-22\transformed_data\train\insurance.npz'
test_array_path=r'D:\Python_projects\Insurance_Prediction\InsurancePremiumPrediction\artifact\data_transformation\22-07-19-07-52-22\transformed_data\test\insurance.npz'
train_array=load_numpy_array_data(train_array_path)
test_array=load_numpy_array_data(test_array_path)
x_train,y_train,x_test,y_test=train_array[:,:-1],train_array[:,-1],test_array[:,:-1],test_array[:,-1]

In [50]:
model_dict,model_score_set=training_models(x_train,y_train,x_test,y_test)


In [51]:
model_dict

{'linear_model': LinearRegression(),
 'decision_tree_model': DecisionTreeRegressor(),
 'random_forest': RandomForestRegressor(),
 'gradient_booster': GradientBoostingRegressor()}

In [52]:
model_score_set

{LinearRegression(): [0.7564369419272546, 0.7215214663841498],
 DecisionTreeRegressor(): [0.9983784082272196, 0.6232858674394207],
 RandomForestRegressor(): [0.9778219591427031, 0.805583422045145],
 GradientBoostingRegressor(): [0.9081263946076713, 0.8217932058703239]}

In [53]:
def training_models(x_train,y_train,x_test,y_test):
        try:
            X_train,Y_train,X_test,Y_test=x_train,y_train,x_test,y_test

            model_dict={}
            linear_model=LinearRegression()
            decision_tree_model=DecisionTreeRegressor()
            random_forest=RandomForestRegressor()
            gradient_booster=GradientBoostingRegressor()

            model_dict['linear_model']=linear_model
            model_dict['decision_tree_model']=decision_tree_model
            model_dict['random_forest']=random_forest
            model_dict['gradient_booster']=gradient_booster

            model_score_set={}
            for i in model_dict.values():
                i.fit(X_train,Y_train)
                Y_predict=i.predict(X_test)
                model_score_set[i]=[i.score(X_train,Y_train),r2_score(Y_test,Y_predict)]
            return model_dict,model_score_set
        except Exception as e:
            raise Exception

In [54]:
y_train

array([ 6986.7 , 20234.85,  1719.44, ...,  3732.63,  1842.52, 40932.43])

In [55]:
y_train.shape

(1070,)